In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/Data

/content/drive/MyDrive/Data


In [3]:
import pandas as pd
non_PRC = pd.read_csv('eye_open/50%/eye_open50.csv')
PRC = pd.read_csv('eye_open/50%/eye_open_par50.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [4]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import warnings
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [5]:
score = clinical_assessment.iloc[:,9]

In [6]:
score = score.dropna()
del score[0]
del score[15]

In [7]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


## Non-Paretic Side

In [8]:
df_non = pd.DataFrame(non_PRC)
y = pd.DataFrame(y)

In [9]:
non = pd.concat([y,df_non],axis=1)
colname = list(non.columns)
colname[0] = "score"
non.columns = colname
non.columns

Index(['score', 'Unnamed: 0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7',
       'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16',
       'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25',
       'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34',
       'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43'],
      dtype='object')

In [10]:
X = non.loc[:,['PC5','PC6','PC15','PC16','PC18','PC20','PC23','PC24','PC31']]
y = non['score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123, stratify=y)

### Logistic Regression

In [11]:
lr_model = LogisticRegression()

# 선택된 특성으로 모델 학습
lr_model.fit(X_train, y_train)

accuracy_scores = cross_val_score(lr_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(lr_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(lr_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(lr_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6668
Average Precision: 0.8
Average F1-score: 0.7333999999999999
Average AUC: 0.7


### Random Forest

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6668
Average Precision: 0.6668
Average F1-score: 0.7962
Average AUC: 0.5833999999999999


### SVM Linear

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.65
Average Precision: 0.7666000000000001
Average F1-score: 0.7134
Average AUC: 0.6


### SVM Poly

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6834
Average Precision: 0.7
Average F1-score: 0.7827999999999999
Average AUC: 0.75


### XGBoost

In [15]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6167999999999999
Average Precision: 0.6834
Average F1-score: 0.7314
Average AUC: 0.7834


## Paretic Side

In [16]:
df_par = pd.DataFrame(PRC)
y = pd.DataFrame(y)
par = pd.concat([y,df_par],axis=1)
colname = list(par.columns)
colname[0] = "score"

In [17]:
import random
random.seed(1)
X = par.loc[:,['PC2','PC10','PC13','PC16','PC18','PC26','PC28','PC30','PC40','PC42']]
y = par['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123,stratify=y)

### Logistic Regression

In [18]:
lr_model = LogisticRegression()

# 선택된 특성으로 모델 학습
lr_model.fit(X_train, y_train)

accuracy_scores = cross_val_score(lr_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(lr_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(lr_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(lr_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7333999999999999
Average Precision: 0.8
Average F1-score: 0.8096
Average AUC: 0.8333999999999999


### Random Forest

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6668
Average Precision: 0.6668
Average F1-score: 0.7962
Average AUC: 0.6334


### SVM Linear

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6668
Average Precision: 0.7834
Average F1-score: 0.7581999999999999
Average AUC: 0.6334


### SVM Poly

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.4666
Average Precision: 0.5
Average F1-score: 0.5762
Average AUC: 0.1666


### XGBoost

In [22]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7333999999999999
Average Precision: 0.7333999999999999
Average F1-score: 0.8362
Average AUC: 0.625
